In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def weighted_divisor_sum(k,r):
    ans=0
    pfk=prime_factors(k)
    weights=[]
    for d in divisors(k):
        weight=1
        pfd=prime_factors(d)
        for p in pfd:
            opd=order(p,d)
            opk=order(p,k)
            wt=binomial(opk,opd)
            weight=weight*wt
        weights=weights+[weight]
        ans=ans+weight*d^r
    return ans

def prime_factors(n):
    F=factor(n)
    f0=[f[0] for f in F]
    return sortAndSmash(f0)

def omega(n):
    if n==1:return 1
    if n>1:return len(prime_factors(n))

def omega2(n):
    return len(prime_factors(n))

def bigOmega(n):
    F=factor(n)
    ans=0
    for f in F:
        ans=ans+f[1]
    return ans

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [3]:
wtDvSrs=0
for n in [1..10]:
    wtDvSrs=wtDvSrs+weighted_divisor_sum(n,1)*x^n
print(wtDvSrs)
print(wtDvSrs.truncate(4))

18*x^10 + 16*x^9 + 27*x^8 + 8*x^7 + 12*x^6 + 6*x^5 + 9*x^4 + 4*x^3 + 3*x^2 + x
4*x^3 + 3*x^2 + x


In [4]:
for r in [0..12]:
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..30]:
        c=recip_wtDvPowerConstant(k)
        o=order(2,c)
        data=data+[o]
    
    print("r:",r);print(data);print()

r: 0
[1, 3, 2, 4, 2, 2, 3, 4, 2, 2, 3, 4, 2, 3, 4, 5, 4, 2, 3, 3, 2, 2, 4, 4, 2, 8, 3, 3, 6, 2]

r: 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 4
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 5
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 6
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 7
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 8
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 9
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 10
[0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
for t in [0..12]:
    r=2*t+1
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..100]:
        c=recip_wtDvPowerConstant(2*k)
        o=order(3,c)
        sk=c/3^o
        md=mod(sk,3)
        data=data+[(o,md)]
    
    print("r:",r);print(data);print()

r: 1
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

r: 3
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0,

r: 23
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]

r: 25
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (

In [10]:
for r in [0..12]:
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..100]:
        c=recip_wtDvPowerConstant(k)
        o=order(3,c)
        data=data+[o]
    
    print("r:",r);print(data);print()

r: 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0]

r: 1
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 3, 0, 2, 0, 6, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 4, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 3, 0, 2, 0, 1, 0, 3, 0, 1, 0, 2, 0, 2, 0, 3, 0, 1, 0, 2, 0, 3, 0, 2, 0, 6, 0, 2, 0, 2, 0, 3, 0, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 3, 0]

r: 2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 3, 0, 0]

r: 3
[2, 0, 3, 0, 2, 0, 4, 0, 4, 0, 2, 0, 2, 0, 3, 0, 2, 0, 3, 0, 3, 0, 3, 0, 4

In [4]:
for r in [5..50]:
    if is_prime(r):
        data=[]
        wtDvSrs=0
        for n in [1..1000]:
            wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
        def recip_wtDv(n):
            g=L(1/wtDvSrs.truncate(2*n))
            return g

        def recip_wtDvPowerConstant(n):
            f=recip_wtDv(n+1)^n
            return f[0]
    
        for k in [1..30]:
            c=recip_wtDvPowerConstant(k)
            o=order(3,c)
            data=data+[o]
    
        print("r:",r);print(data);print()

r: 5
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 7
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 10, 0, 1, 0]

r: 11
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 13
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 17
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 7, 0, 1, 0]

r: 19
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 23
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 29
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 31
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 37
[1, 0, 2, 0, 1, 0, 2, 0, 3, 0, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 6, 0, 1, 0]

r: 41
[1, 0, 2, 0, 1,

In [7]:
data=[]
for r in [2..100]:
    if is_prime(r):
        
        wtDvSrs=0
        for n in [1..1000]:
            wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
        def recip_wtDv(n):
            g=L(1/wtDvSrs.truncate(2*n))
            return g

        def recip_wtDvPowerConstant(n):
            f=recip_wtDv(n+1)^n
            return f[0]
    
        for k in [27..27]:
            c=recip_wtDvPowerConstant(k)
            o=order(3,c)
            data=data+[o]
print(data);print()

[0, 6, 6, 10, 6, 6, 7, 6, 6, 6, 6, 6, 6, 7, 6, 7, 6, 8, 6, 7, 6, 7, 6, 9, 7]



In [7]:
for r in [0..12]:
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..30]:
        c=recip_wtDvPowerConstant(k)
        o=order(5,c)
        data=data+[o]
    
    print("r:",r);print(data);print()

r: 0
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

r: 1
[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

r: 2
[1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 1, 0, 1, 1, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 1, 0, 1, 1]

r: 3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

r: 4
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1]

r: 5
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

r: 6
[1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 1, 0, 1, 2, 2, 0, 1, 0, 2, 0, 2, 0, 2, 0, 3, 0, 1, 0, 1, 1]

r: 7
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

r: 8
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]

r: 9
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

r: 10
[2, 2, 4, 0, 3, 2, 4, 0,

In [9]:
for r in [0..12]:
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..30]:
        c=recip_wtDvPowerConstant(k)
        o=order(7,c)
        data=data+[o]
    
    print("r:",r);print(data);print()

r: 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 1
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]

r: 2
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

r: 3
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 4
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]

r: 5
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

r: 6
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]

r: 7
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]

r: 8
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

r: 9
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

r: 10
[0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
for r in [0..12]:
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..30]:
        c=recip_wtDvPowerConstant(k)
        o=order(3,c)
        sk=c/3^o
        md=mod(sk,3)
        data=data+[md]
    
    print("r:",r);print(data);print()

r: 0
[1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2]

r: 1
[2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1]

r: 2
[1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2]

r: 3
[2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1]

r: 4
[1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2]

r: 5
[1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1]

r: 6
[1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2]

r: 7
[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1]

r: 8
[1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2]

r: 9
[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1]

r: 10
[1, 1, 1, 2, 1, 1, 2, 1,

In [12]:
for r in [0..12]:
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..30]:
        c=recip_wtDvPowerConstant(k)
        o=order(5,c) 
        sk=c/5^o
        md=mod(sk,5)
        data=data+[md]
    
    print("r:",r);print(data);print()

r: 0
[3, 3, 1, 2, 3, 2, 2, 4, 1, 3, 2, 1, 3, 1, 1, 3, 2, 3, 2, 2, 1, 2, 3, 4, 3, 3, 4, 2, 1, 2]

r: 1
[2, 4, 2, 1, 2, 3, 4, 3, 2, 4, 1, 3, 2, 4, 2, 4, 1, 2, 2, 1, 4, 4, 2, 1, 2, 4, 3, 2, 2, 3]

r: 2
[4, 1, 1, 1, 1, 4, 3, 1, 1, 1, 4, 1, 2, 1, 3, 1, 3, 4, 2, 1, 4, 2, 4, 1, 3, 4, 2, 3, 4, 3]

r: 3
[1, 2, 1, 1, 1, 4, 1, 1, 1, 2, 4, 2, 3, 2, 1, 2, 2, 3, 1, 1, 1, 2, 1, 1, 1, 3, 3, 3, 1, 4]

r: 4
[3, 3, 1, 1, 3, 1, 4, 2, 3, 3, 4, 1, 4, 3, 1, 2, 3, 4, 1, 1, 4, 3, 2, 1, 3, 2, 1, 1, 3, 2]

r: 5
[2, 4, 2, 1, 2, 3, 1, 3, 2, 4, 1, 3, 2, 4, 2, 4, 1, 2, 2, 1, 4, 3, 2, 1, 2, 4, 3, 2, 2, 3]

r: 6
[2, 3, 4, 1, 3, 2, 2, 1, 3, 1, 2, 1, 1, 1, 4, 1, 4, 4, 2, 1, 3, 2, 1, 1, 4, 4, 1, 3, 2, 3]

r: 7
[1, 2, 1, 1, 1, 4, 1, 1, 1, 2, 4, 2, 3, 2, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 4]

r: 8
[3, 3, 1, 1, 3, 3, 4, 2, 3, 3, 2, 1, 4, 3, 1, 2, 3, 4, 1, 1, 4, 3, 2, 1, 3, 2, 1, 1, 3, 2]

r: 9
[2, 4, 2, 1, 2, 3, 3, 3, 2, 4, 1, 3, 2, 4, 2, 4, 1, 2, 2, 1, 4, 3, 2, 1, 2, 4, 3, 2, 2, 3]

r: 10
[4, 1, 1, 1, 1, 4, 3, 1,

In [13]:
for r in [0..12]:
    data=[]
    wtDvSrs=0
    for n in [1..1000]:
        wtDvSrs=wtDvSrs+weighted_divisor_sum(n,r)*x^n
    
    def recip_wtDv(n):
        g=L(1/wtDvSrs.truncate(2*n))
        return g

    def recip_wtDvPowerConstant(n):
        f=recip_wtDv(n+1)^n
        return f[0]
    
    for k in [1..30]:
        c=recip_wtDvPowerConstant(k)
        o=order(7,c) 
        sk=c/7^o
        md=mod(sk,7)
        data=data+[md]
    
    print("r:",r);print(data);print()

r: 0
[5, 1, 5, 6, 4, 2, 5, 4, 4, 1, 6, 4, 3, 1, 5, 3, 6, 1, 3, 2, 5, 2, 6, 1, 6, 3, 3, 6, 2, 2]

r: 1
[4, 5, 1, 4, 6, 1, 4, 3, 4, 3, 2, 6, 4, 5, 3, 3, 2, 5, 6, 5, 1, 4, 2, 4, 3, 6, 1, 5, 3, 3]

r: 2
[2, 6, 3, 2, 6, 1, 2, 6, 2, 6, 6, 2, 2, 6, 6, 6, 5, 6, 3, 6, 3, 6, 5, 1, 3, 1, 3, 2, 5, 6]

r: 3
[5, 5, 6, 6, 4, 1, 5, 4, 6, 5, 6, 1, 5, 5, 4, 6, 3, 2, 5, 5, 6, 5, 1, 4, 4, 5, 6, 6, 2, 5]

r: 4
[4, 3, 2, 3, 5, 1, 4, 3, 6, 5, 3, 3, 4, 3, 1, 2, 4, 3, 3, 3, 2, 2, 2, 6, 5, 1, 2, 3, 6, 4]

r: 5
[2, 5, 1, 4, 1, 1, 2, 1, 2, 2, 5, 4, 2, 3, 5, 5, 4, 3, 4, 3, 1, 2, 5, 5, 4, 2, 1, 4, 2, 2]

r: 6
[5, 1, 5, 6, 4, 1, 5, 5, 6, 5, 2, 1, 5, 1, 1, 2, 2, 1, 2, 1, 5, 5, 6, 1, 1, 6, 5, 6, 4, 4]

r: 7
[4, 5, 1, 2, 6, 1, 4, 3, 5, 3, 2, 6, 4, 5, 3, 3, 5, 5, 1, 5, 1, 4, 2, 4, 3, 6, 1, 5, 5, 3]

r: 8
[2, 6, 3, 2, 6, 1, 2, 6, 2, 6, 6, 5, 2, 6, 6, 6, 5, 6, 3, 6, 3, 3, 5, 1, 3, 1, 3, 2, 5, 6]

r: 9
[5, 5, 6, 6, 6, 1, 5, 4, 6, 5, 6, 1, 5, 5, 4, 6, 3, 2, 5, 5, 6, 5, 1, 4, 4, 5, 6, 6, 2, 5]

r: 10
[4, 3, 2, 3, 5, 1, 4, 3,